In [6]:
#!/usr/bin/env python3
"""
統合NMC-Al界面構造生成・最適化システム
リチウムイオン電池リサイクル研究用

完全統合版: 構造生成 → 表面作成 → 界面構築 → トリミング → 最適化

Features:
- NMC組成変動構造の自動生成
- 体系的な表面・界面構築
- Trajectoryファイルの自動整理
- 界面構造のトリミングと最適化
- 複数ミラー指数対応
- エラー耐性とロバスト処理
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os, sys, time
from pathlib import Path
import random
import math
from collections import Counter

# ASE
import ase
from ase import Atoms, units
from ase.io import read, write
from ase.build import surface, molecule, add_adsorbate, fcc100, stack
from ase.constraints import FixAtoms, FixedPlane, FixBondLength, ExpCellFilter, UnitCellFilter
from ase.optimize import BFGS, LBFGS, FIRE
from ase.visualize import view

# PFP (Matlantis) - 統合版
try:
    # 基本PFP
    from pfp_api_client.pfp.calculators.ase_calculator import ASECalculator
    from pfp_api_client.pfp.estimator import Estimator
    from pfp_api_client.pfp.estimator import EstimatorCalcMode
    
    # Matlantis機能
    from matlantis_features.atoms import MatlantisAtoms
    from matlantis_features.features.common.opt import FireLBFGSASEOptFeature
    from matlantis_features.utils.calculators import pfp_estimator_fn
    
    # 計算器初期化
    estimator = Estimator(calc_mode='CRYSTAL_U0_PLUS_D3')
    calculator = ASECalculator(estimator)
    MATLANTIS_AVAILABLE = True
    print("✅ PFP/Matlantis計算器を正常に初期化しました")
    
except ImportError as e:
    print(f"❌ PFP/Matlantisが利用できません: {e}")
    print("  基本ASE最適化のみを使用します")
    calculator = None
    MATLANTIS_AVAILABLE = False

class IntegratedNMCInterfaceSystem:
    """統合NMC-Al界面構造システム"""
    
    def __init__(self, base_dir="/home/jovyan/Kaori/MD/LiB_2"):
        self.base_dir = Path(base_dir)
        self.input_dir = Path("/home/jovyan/Kaori/MD/input")
        self.output_dir = self.base_dir / "structure" / "output"
        self.interfaces_dir = self.output_dir / "final_interfaces2"
        
        # ディレクトリ作成
        for dir_path in [self.input_dir, self.output_dir, self.interfaces_dir]:
            dir_path.mkdir(parents=True, exist_ok=True)
        
        # 計算器設定
        self.calculator = calculator
        
        # Miller指数リスト（重要度順）
        self.miller_indices = [
            (0, 0, 1),  # NMC(001) - 最安定面、プレート形状
            (1, 0, 4),  # NMC(104) - 中活性、多面体形状
            (0, 1, 2),  # NMC(012) - 高活性、切頭八面体形状
            (0, 1, 0),  # NMC(010) - Li拡散活性面
            (1, 1, 1),  # Al(111) - Al最安定面
            (1, 0, 0),  # Al(100), NMC(100) - 基本面
            (1, 1, 0),  # Al(110), NMC(110) - 副次面
        ]
        
        print(f"✅ 統合システム初期化完了")
        print(f"  ベースディレクトリ: {self.base_dir}")
        print(f"  出力ディレクトリ: {self.output_dir}")
        print(f"  界面保存先: {self.interfaces_dir}")
    
    # =================== PHASE 1: 構造生成フェーズ ===================
    
    def create_nmc_structure(self):
        """NMC111基本構造の作成（CIFファイルがない場合の代替）"""
        print("NMC111基本構造を作成中...")
        
        # 層状岩塩構造のNMC111を近似的に作成
        # Li層、遷移金属層、酸素層の積層構造
        a = 2.87  # 格子定数 (Å)
        c = 14.2  # c軸方向格子定数
        
        # 基本セルの原子位置
        positions = [
            # Li層 (z=0)
            [0.0, 0.0, 0.0],      # Li
            [0.5, 0.5, 0.0],      # Li
            
            # 遷移金属層 (z=0.25c)
            [0.0, 0.5, 0.25],     # Ni
            [0.5, 0.0, 0.25],     # Mn
            
            # 酸素層 (z=0.125c, z=0.375c)
            [0.0, 0.0, 0.125],    # O
            [0.5, 0.5, 0.125],    # O
            [0.0, 0.5, 0.375],    # O
            [0.5, 0.0, 0.375],    # O
            
            # 遷移金属層 (z=0.5c)
            [0.0, 0.0, 0.5],      # Co
            [0.5, 0.5, 0.5],      # Ni
            
            # Li層 (z=0.75c)
            [0.0, 0.5, 0.75],     # Li
            [0.5, 0.0, 0.75],     # Li
        ]
        
        symbols = ['Li', 'Li', 'Ni', 'Mn', 'O', 'O', 'O', 'O', 'Co', 'Ni', 'Li', 'Li']
        
        cell = [[a, 0, 0], [0, a, 0], [0, 0, c]]
        
        nmc_atoms = Atoms(symbols=symbols, 
                         positions=np.array(positions) * [a, a, c],
                         cell=cell, 
                         pbc=True)
        
        print(f"✅ NMC111構造作成完了: {len(nmc_atoms)} atoms")
        return nmc_atoms
    
    def load_or_create_nmc_base(self):
        """NMC基本構造の読み込みまたは作成"""
        cif_path = self.input_dir / "Li3MnCoNiO6.cif"
        
        if cif_path.exists():
            print(f"CIFファイルから読み込み: {cif_path}")
            try:
                base_atoms = read(str(cif_path))
                print(f"✅ CIF読み込み成功: {len(base_atoms)} atoms")
                return base_atoms
            except Exception as e:
                print(f"❌ CIF読み込み失敗: {e}")
        
        print("CIFファイルがないため、基本構造を生成します")
        return self.create_nmc_structure()
    
    def optimize_structure_systematic(self, atoms, name="structure"):
        """FireLBFGS最適化（Matlantis使用）"""
        print(f"=== {name}のMatlantis最適化開始 ===")
        
        if not MATLANTIS_AVAILABLE:
            print("Matlantisが利用できないため、最適化をスキップします")
            return atoms
        
        # FireLBFGS最適化を実行
        trajectory_path = self.output_dir / f"{name}_optimization.traj"
        optimized_atoms = self.run_matlantis_optimization(
            atoms, trajectory_path, fmax=0.05, name=name
        )
        
        return optimized_atoms if optimized_atoms is not None else atoms
    
    def create_structure_from_target(self, base_atoms, target_composition, transition_metals):
        """目標組成から新しい構造を作成"""
        if not math.isclose(sum(target_composition.values()), 1.0):
            raise ValueError("目標組成の合計が1.0になりません。")
    
        variant_atoms = base_atoms.copy()
        tm_indices = [i for i, atom in enumerate(variant_atoms) if atom.symbol in transition_metals]
        total_tm_sites = len(tm_indices)
    
        if total_tm_sites == 0:
            print(f"警告: 置換対象の元素 {transition_metals} が見つかりません")
            return None
    
        # 目標原子数の計算
        target_counts = {symbol: round(fraction * total_tm_sites) 
                        for symbol, fraction in target_composition.items()}
    
        # 四捨五入による過不足を調整
        current_total = sum(target_counts.values())
        diff = total_tm_sites - current_total
        if diff != 0:
            most_abundant_element = max(target_composition, key=target_composition.get)
            target_counts[most_abundant_element] += diff
    
        # 新しい元素リストを作成
        new_symbols = []
        for symbol, count in target_counts.items():
            new_symbols.extend([symbol] * count)
    
        # ランダムに原子を再配置
        random.seed(42)
        random.shuffle(tm_indices)
    
        for i, atom_index in enumerate(tm_indices):
            variant_atoms[atom_index].symbol = new_symbols[i]
    
        return variant_atoms

    def create_composition_variants(self, base_atoms, base_name="NMC"):
        """組成変動構造の作成"""
        print(f"=== {base_name}の組成変動構造作成 ===")
        
        variants = {f"{base_name}_pristine": base_atoms.copy()}
        
        # シミュレーション計画の定義（簡略版）
        simulation_plan = [
            # ニッケル(Ni)増減の影響
            {"name": "Ni_increase", "composition": {"Ni": 0.6, "Mn": 0.2, "Co": 0.2}},
            {"name": "Ni_decrease", "composition": {"Ni": 0.2, "Mn": 0.4, "Co": 0.4}},
            
            # マンガン(Mn)増減の影響
            {"name": "Mn_increase", "composition": {"Ni": 0.2, "Mn": 0.6, "Co": 0.2}},
            {"name": "Mn_decrease", "composition": {"Ni": 0.4, "Mn": 0.2, "Co": 0.4}},
            
            # コバルト(Co)増減の影響
            {"name": "Co_increase", "composition": {"Ni": 0.2, "Mn": 0.2, "Co": 0.6}},
            {"name": "Co_decrease", "composition": {"Ni": 0.4, "Mn": 0.4, "Co": 0.2}},
            
            # ハイニッケル化トレンドの評価
            {"name": "NMC523_approx", "composition": {"Ni": 0.50, "Mn": 0.25, "Co": 0.25}},
            {"name": "NMC622_approx", "composition": {"Ni": 0.60, "Mn": 0.20, "Co": 0.20}},
            {"name": "NMC811_approx", "composition": {"Ni": 0.80, "Mn": 0.10, "Co": 0.10}},
        ]
        
        for target in simulation_plan:
            name = target["name"]
            composition = target["composition"]
            
            try:
                variant_atoms = self.create_structure_from_target(
                    base_atoms, composition, ["Ni", "Mn", "Co"])
                
                if variant_atoms is not None:
                    variant_name = f"{base_name}_{name}"
                    variants[variant_name] = variant_atoms
                    print(f"✅ 作成完了: {variant_name}")
                    
            except Exception as e:
                print(f"❌ エラー: {name} - {e}")
        
        return variants
    
    def makesurface_advanced(self, atoms, miller_indices=(1,0,0), layers=4, 
                           rep=[4,4,1], vacuum=15.0, fix_ratio=0.3, name="surface"):
        """改良版表面作成関数"""
        print(f"表面作成: {name} {miller_indices}面, {layers}層")
        
        try:
            slab = surface(atoms, miller_indices, layers, vacuum=vacuum)
            slab = slab.repeat(rep)
            
            # 位置調整
            positions = slab.get_positions()
            z_min = positions[:, 2].min()
            slab.set_positions(positions - [0, 0, z_min])
            
            # 下層原子の固定
            z_coords = slab.positions[:, 2]
            z_max = z_coords.max()
            z_threshold = z_coords.min() + (z_max - z_coords.min()) * fix_ratio
            fixed_indices = [i for i, z in enumerate(z_coords) if z <= z_threshold]
            
            if fixed_indices:
                constraint = FixAtoms(indices=fixed_indices)
                slab.set_constraint(constraint)
                print(f"固定原子数: {len(fixed_indices)}/{len(slab)}")
            
            # 構造最適化
            slab = self.optimize_structure_systematic(slab, f"{name}")
            
            return slab
        except Exception as e:
            print(f"表面作成エラー: {e}")
            return None
    
    def prepare_al_slab(self, size=(6, 6, 10), vacuum=5.0):
        """Al(100)スラブモデルを作成"""
        slab_al = fcc100("Al", size=size, orthogonal=True, periodic=True, vacuum=vacuum)
        print(f"Al(100)スラブ作成: サイズ={size}")
        return slab_al
    
    def build_al_structures(self):
        """Al構造の構築"""
        print("=== Al構造構築 ===")
        
        al_bulk = self.prepare_al_slab(size=(5,5,10), vacuum=1.0)
        
        al_surfaces = {}
        for miller in self.miller_indices[:3]:  # Al主要面のみ
            surface_name = f"Al_{miller[0]}{miller[1]}{miller[2]}"
            al_slab = self.makesurface_advanced(al_bulk, miller_indices=miller, 
                                              name=surface_name)
            if al_slab is not None:
                al_surfaces[surface_name] = al_slab
                
        return al_bulk, al_surfaces
    
    # =================== PHASE 2: ファイル整理フェーズ ===================
    
    def cleanup_trajectory_files(self):
        """Trajectoryファイルの整理"""
        print("\n--- Trajectoryファイルの分析と整理を開始 ---")
        
        traj_files = list(self.output_dir.glob("*.traj"))
        print(f"🔍 {len(traj_files)} 個の .traj ファイルを発見")
        
        for traj_path in traj_files:
            try:
                file_size = traj_path.stat().st_size
                if file_size == 0:
                    print(f"  -> 🗑️  0KBファイルを削除: {traj_path.name}")
                    traj_path.unlink()
                    continue

                # 正常なファイルは変換
                base_name = traj_path.stem
                initial_xyz_path = self.output_dir / f"{base_name}_initial.xyz"
                final_xyz_path = self.output_dir / f"{base_name}_final.xyz"

                print(f"  -> 🔄 変換中: {traj_path.name}")

                atoms_list = read(str(traj_path), index=":")
                if not atoms_list:
                    print(f"  -> ❌ 空のTrajectory、削除: {traj_path.name}")
                    traj_path.unlink()
                    continue

                # .xyzファイルとして保存
                write(str(initial_xyz_path), atoms_list[0])
                write(str(final_xyz_path), atoms_list[-1])

                # 元の.trajファイルを削除
                traj_path.unlink()
                print(f"  -> ✔️ 変換成功: {traj_path.name}")

            except Exception as e:
                print(f"  -> ❌ エラー ({traj_path.name}): {e}")

        print("✨ Trajectory整理完了")
    
    # =================== PHASE 3: 界面構築・最適化フェーズ ===================
    
    def run_matlantis_optimization(self, atoms, trajectory_path, fmax=0.05, name="structure"):
        """Matlantis最適化"""
        print(f"  -> Matlantis最適化開始 ({name}, fmax = {fmax}) ...")
        
        if not MATLANTIS_AVAILABLE:
            print(f"  -> ⚠️ Matlantisが利用できないため、最適化をスキップします")
            return atoms
        
        try:
            matlantis_atoms = MatlantisAtoms(atoms)
            estimator_function = pfp_estimator_fn(
                model_version='v7.0.0', calc_mode=EstimatorCalcMode.CRYSTAL_PLUS_D3
            )
            position_optimizer = FireLBFGSASEOptFeature(
                estimator_fn=estimator_function, filter=False, 
                trajectory=str(trajectory_path),
                n_run=5000, fmax=fmax, show_progress_bar=True
            )
            
            result = position_optimizer(matlantis_atoms)
            optimized_atoms = result.atoms.ase_atoms
            final_energy = result.output.energy_log[-1]
            print(f"  -> ✔️ 最適化完了！ エネルギー: {final_energy:.3f} eV")
            
            # .xyzファイルでの保存
            xyz_save_path = Path(trajectory_path).with_suffix('.xyz')
            write(str(xyz_save_path), optimized_atoms)
            return optimized_atoms
            
        except Exception as e:
            print(f"  -> ❌ 最適化中にエラーが発生: {e}")
            return None


    def build_interface(self, slab1, slab2, target_xy=(12.0, 12.0), separation=2.0):
        """界面構築"""
        print(f"    1. 界面を構築中...")
        
        # XY方向のトリミング
        position1 = slab1.get_positions()
        position2 = slab2.get_positions()
        indices_to_delete1 = np.where((position1[:, 0] >= target_xy[0]) | 
                                     (position1[:, 1] >= target_xy[1]))[0]
        indices_to_delete2 = np.where((position2[:, 0] >= target_xy[0]) | 
                                     (position2[:, 1] >= target_xy[1]))[0]
        
        cut_slab1, cut_slab2 = slab1.copy(), slab2.copy()
        if len(indices_to_delete1) > 0: 
            del cut_slab1[indices_to_delete1]
        if len(indices_to_delete2) > 0: 
            del cut_slab2[indices_to_delete2]
        
        # Z方向の位置調整
        z1_max = cut_slab1.positions[:, 2].max()
        z2_min = cut_slab2.positions[:, 2].min()
        cut_slab2.positions[:, 2] += z1_max - z2_min + separation
        
        # 界面の結合
        interface = cut_slab1 + cut_slab2
        interface.cell = [12, 12, 53]
        interface.pbc = [True, True, False]
        
        return interface

    def trim_interface(self, atoms, thickness_per_slab=25.0):
        """界面のトリミング"""
        print(f"    2. 界面をトリミング中 (各スラブ厚み: {thickness_per_slab} Å)...")
        
        # 界面のZ座標を推定
        al_indices = [a.index for a in atoms if a.symbol == 'Al']
        nmc_indices = [a.index for a in atoms if a.symbol not in ['Al']]
        
        if not al_indices or not nmc_indices: 
            return None
            
        al_pos = atoms.get_positions()[al_indices]
        nmc_pos = atoms.get_positions()[nmc_indices]
        
        if np.mean(al_pos[:, 2]) < np.mean(nmc_pos[:, 2]):
            interface_z = (al_pos[:, 2].max() + nmc_pos[:, 2].min()) / 2
            is_al_bottom = True
        else:
            interface_z = (nmc_pos[:, 2].max() + al_pos[:, 2].min()) / 2
            is_al_bottom = False

        # 削除する原子を特定
        indices_to_delete = []
        for atom in atoms:
            if is_al_bottom:
                if atom.symbol == 'Al' and atom.z < interface_z - thickness_per_slab:
                    indices_to_delete.append(atom.index)
                elif atom.symbol != 'Al' and atom.z > interface_z + thickness_per_slab:
                    indices_to_delete.append(atom.index)
            else:
                if atom.symbol != 'Al' and atom.z < interface_z - thickness_per_slab:
                    indices_to_delete.append(atom.index)
                elif atom.symbol == 'Al' and atom.z > interface_z + thickness_per_slab:
                    indices_to_delete.append(atom.index)

        print(f"       -> {len(indices_to_delete)} 個の原子を削除")
        if len(indices_to_delete) > 0:
            del atoms[indices_to_delete]
        return atoms

    def create_trim_and_optimize_interface(self, slab1_path, slab2_path):
        """界面の作成、トリミング、最適化"""
        slab1_name = slab1_path.stem.replace('_pos_opt_final', '').replace('_cell_opt_final', '')
        slab2_name = slab2_path.stem.replace('_pos_opt_final', '').replace('_cell_opt_final', '')
        interface_name = f"Interface_{slab1_name}_on_{slab2_name}_trimmed"
        output_path = self.interfaces_dir / f"{interface_name}.cif"

        if output_path.exists():
            print(f"✅ 既存ファイルあり、スキップ: {output_path.name}")
            return

        print(f"\n--- 処理開始: {slab1_name} + {slab2_name} ---")
        try:
            # STEP 1: 構築
            slab1, slab2 = read(str(slab1_path)), read(str(slab2_path))
            interface_built = self.build_interface(slab1, slab2)

            # STEP 2: トリミング
            interface_trimmed = self.trim_interface(interface_built, thickness_per_slab=25.0)
            if interface_trimmed is None:
                print("   -> ❌ トリミングに失敗")
                return

            # STEP 3: 最適化
            print(f"    3. トリミング後の構造を最適化中...")
            traj_path = self.interfaces_dir / f"{interface_name}.traj"
            optimized_structure = self.run_matlantis_optimization(
                interface_trimmed, traj_path, fmax=0.05, name=interface_name)

            # STEP 4: 保存
            if optimized_structure:
                optimized_structure.center(vacuum=15.0, axis=2)
                optimized_structure.pbc = (True, True, False)
                print(f"    4. 最終構造をファイルに保存中 -> {output_path.name}")
                write(str(output_path), optimized_structure)
                print(f"✅ 界面構造完成: {interface_name}")
            else:
                print("   -> ❌ 最適化に失敗")

        except Exception as e:
            print(f"  -> ❌ この界面の作成中にエラーが発生: {e}")

    def process_all_interfaces(self):
        """全界面の処理"""
        print("\n--- STEP 3: 界面構築・トリミング・最適化プロセス開始 ---")
        
        al_slabs, cathode_slabs = self.find_and_categorize_surfaces()
        if not al_slabs or not cathode_slabs:
            print(" -> 表面ファイルが不足しているため、スキップします。")
            return

        print(f"\n--- Al/NMC界面の組み合わせ処理を開始 ---")
        total_combinations = len(al_slabs) * len(cathode_slabs)
        current = 0
        
        for al_path in al_slabs:
            for cathode_path in cathode_slabs:
                current += 1
                print(f"\n進行状況: {current}/{total_combinations}")
                self.create_trim_and_optimize_interface(al_path, cathode_path)

        print("\n✨ 全ての界面処理が完了")
    
    # =================== メイン実行フェーズ ===================
    
    def save_structures(self, structures, phase_name="structures"):
        """構造の保存"""
        print(f"\n=== {phase_name}の保存 ===")
        
        for name, atoms in structures.items():
            try:
                if atoms is not None:
                    filepath = self.output_dir / f"{name}.traj"
                    write(str(filepath), atoms)
                    print(f"保存: {name}")
            except Exception as e:
                print(f"保存失敗: {name} - {e}")
        
        print(f"保存完了: {len(structures)}個の構造")

    def generate_final_summary(self):
        """最終サマリーの生成"""
        print("\n" + "=" * 70)
        print("🎯 統合NMC-Al界面構造システム 実行結果サマリー")
        print("=" * 70)
        
        # ファイル数のカウント
        cif_files = list(self.interfaces_dir.glob("*.cif"))
        xyz_files = list(self.output_dir.glob("*.xyz"))
        traj_files = list(self.output_dir.glob("*.traj"))
        
        print(f"\n📊 生成されたファイル:")
        print(f"  - 最終界面構造 (CIF): {len(cif_files)}個")
        print(f"  - 中間構造 (XYZ): {len(xyz_files)}個")
        print(f"  - Trajectoryファイル: {len(traj_files)}個")
        
        if cif_files:
            print(f"\n📋 生成された界面構造リスト:")
            for i, cif_file in enumerate(cif_files[:10], 1):
                file_size = cif_file.stat().st_size / 1024  # KB
                print(f"  {i:2d}. {cif_file.name} ({file_size:.1f} KB)")
            
            if len(cif_files) > 10:
                print(f"     ... (他 {len(cif_files) - 10} 個)")
        
        print(f"\n🔬 解析されたNMC組成バリエーション:")
        print(f"  - 基本NMC111構造")
        print(f"  - Ni増減バリアント (高活性/低活性)")
        print(f"  - Mn増減バリアント (構造安定性)")
        print(f"  - Co増減バリアント (電気化学特性)")
        print(f"  - ハイニッケル化トレンド (NMC523/622/811)")
        
        print(f"\n⚙️ 処理されたMiller指数:")
        miller_descriptions = {
            (0, 0, 1): "最安定面、プレート形状",
            (1, 0, 4): "中活性、多面体形状",
            (0, 1, 2): "高活性、切頭八面体形状",
            (0, 1, 0): "Li拡散活性面",
            (1, 1, 1): "Al最安定面",
            (1, 0, 0): "基本面",
            (1, 1, 0): "副次面"
        }
        for miller in self.miller_indices:
            desc = miller_descriptions.get(miller, "")
            print(f"  - {miller} : {desc}")
        
        print(f"\n🎯 次のステップ推奨事項:")
        print(f"  1. 界面接着強度の定量評価")
        print(f"  2. カレンダリング圧力下でのシミュレーション")
        print(f"  3. 熱処理プロセス条件の最適化")
        print(f"  4. 機械的分離効率の評価")
        print(f"  5. リサイクル歩留まりの予測")
        
        # サマリーをファイルに保存
        summary_text = []
        summary_text.append("統合NMC-Al界面構造システム 実行結果")
        summary_text.append("=" * 50)
        summary_text.append(f"実行日時: {time.strftime('%Y-%m-%d %H:%M:%S')}")
        summary_text.append(f"")
        summary_text.append(f"生成されたファイル:")
        summary_text.append(f"- 最終界面構造 (CIF): {len(cif_files)}個")
        summary_text.append(f"- 中間構造 (XYZ): {len(xyz_files)}個")
        summary_text.append(f"- Trajectoryファイル: {len(traj_files)}個")
        summary_text.append(f"")
        summary_text.append("処理フェーズ:")
        summary_text.append("1. NMC組成バリアント生成")
        summary_text.append("2. Al/NMC表面構造作成")
        summary_text.append("3. Trajectoryファイル整理")
        summary_text.append("4. 界面構築・トリミング")
        summary_text.append("5. Matlantis最適化")
        summary_text.append("6. 最終構造保存")
        
        summary_path = self.output_dir / "execution_summary.txt"
        with open(summary_path, "w", encoding="utf-8") as f:
            f.write("\n".join(summary_text))
        
        print(f"\n💾 実行サマリーを保存しました: {summary_path}")
        print("=" * 70)
    
#     def run_complete_workflow(self):
#         """完全統合ワークフローの実行"""
#         print("🚀 統合NMC-Al界面構造システム 開始")
#         print("=" * 70)
        
#         start_time = time.time()
        
#         try:
#             # =================== PHASE 1: 構造生成 ===================
#             print("\n🔬 PHASE 1: NMC構造生成フェーズ")
#             print("-" * 50)
            
#             # 基本構造の読み込みまたは作成
#             base_nmc = self.load_or_create_nmc_base()
            
#             # 組成変動構造の作成
#             nmc_variants = self.create_composition_variants(base_nmc, "NMC111")
            
#             # Al構造の構築
#             al_bulk, al_surfaces = self.build_al_structures()
            
#             # 全構造の統合
#             all_structures = {"Al_bulk": al_bulk}
#             all_structures.update(nmc_variants)
#             all_surfaces = al_surfaces.copy()
            
#             # NMC表面の作成
#             print("\n=== NMC表面構造作成 ===")
#             for variant_name, atoms in nmc_variants.items():
#                 for miller in self.miller_indices[:4]:  # NMC主要面
#                     try:
#                         surface_name = f"{variant_name}_{miller[0]}{miller[1]}{miller[2]}"
#                         slab = self.makesurface_advanced(atoms, miller_indices=miller,
#                                                        name=surface_name)
#                         if slab is not None:
#                             all_surfaces[surface_name] = slab
#                     except Exception as e:
#                         print(f"表面作成失敗: {surface_name} - {e}")
            
#             # 構造の保存
#             self.save_structures(all_structures, "バルク構造")
#             self.save_structures(all_surfaces, "表面構造")
            
#             print(f"✅ PHASE 1 完了: バルク{len(all_structures)}個, 表面{len(all_surfaces)}個")
            
#             # =================== PHASE 2: ファイル整理 ===================
#             print("\n📁 PHASE 2: Trajectoryファイル整理フェーズ")
#             print("-" * 50)
            
#             self.cleanup_trajectory_files()
#             print("✅ PHASE 2 完了")
            
#             # =================== PHASE 3: 界面最適化 ===================
#             print("\n⚙️ PHASE 3: 界面構築・最適化フェーズ")
#             print("-" * 50)
            
#             self.process_all_interfaces()
#             print("✅ PHASE 3 完了")
            
#             # =================== 最終サマリー ===================
#             execution_time = time.time() - start_time
#             print(f"\n⏱️ 総実行時間: {execution_time:.1f} 秒")
            
#             self.generate_final_summary()
            
#             return True
            
#         except Exception as e:
#             print(f"\n❌ ワークフロー実行中に重大なエラーが発生しました: {e}")
#             print("エラーが発生しても、一部の処理は完了している可能性があります。")
#             print("出力ディレクトリを確認してください。")
#             return False


# def main():
#     """メイン実行関数"""
#     print("=" * 80)
#     print("🧪 統合NMC-Al界面構造生成・最適化システム")
#     print("   リチウムイオン電池リサイクル研究用")
#     print("=" * 80)
    
#     # システム初期化
#     try:
#         system = IntegratedNMCInterfaceSystem()
        
#         # 完全ワークフローの実行
#         success = system.run_complete_workflow()
        
#         if success:
#             print("\n🎉 全てのワークフローが正常に完了しました！")
#             print(f"📂 結果ファイル: {system.interfaces_dir}")
#             print(f"📊 実行ログ: {system.output_dir / 'execution_summary.txt'}")
#         else:
#             print("\n⚠️ 一部のプロセスでエラーが発生しましたが、")
#             print("   完了した処理の結果は保存されています。")
            
#     except Exception as e:
#         print(f"\n💥 システム初期化またはメイン実行でエラーが発生: {e}")
#         print("環境設定または依存関係を確認してください。")
        
#         # 最小限のフォールバック処理
#         print("\n🔄 フォールバック: 基本的なNMC構造のみ生成します...")
#         try:
#             fallback_system = IntegratedNMCInterfaceSystem()
#             base_nmc = fallback_system.load_or_create_nmc_base()
            
#             # 基本構造のみ保存
#             write(str(fallback_system.output_dir / "NMC111_fallback.xyz"), base_nmc)
#             print(f"✅ フォールバック完了: {fallback_system.output_dir / 'NMC111_fallback.xyz'}")
            
#         except Exception as fallback_error:
#             print(f"❌ フォールバック処理も失敗: {fallback_error}")


# if __name__ == "__main__":
#     main()


# # =================== 追加ユーティリティ関数 ===================

def analyze_interface_files(interfaces_dir):
    """界面ファイルの詳細分析"""
    from pathlib import Path
    
    interfaces_path = Path(interfaces_dir)
    if not interfaces_path.exists():
        print(f"ディレクトリが見つかりません: {interfaces_dir}")
        return
    
    print(f"\n🔍 界面構造ファイルの詳細分析")
    print(f"対象ディレクトリ: {interfaces_path}")
    print("-" * 60)
    
    cif_files = list(interfaces_path.glob("*.cif"))
    
    if not cif_files:
        print("CIFファイルが見つかりません。")
        return
    
    composition_analysis = {}
    miller_analysis = {}
    
    for cif_file in cif_files:
        try:
            atoms = read(str(cif_file))
            
            # 組成分析
            symbols = [atom.symbol for atom in atoms]
            composition = Counter(symbols)
            
            # ファイル名からNMC組成とMiller指数を抽出
            filename = cif_file.stem
            
            # NMC組成の抽出
            if "NMC111_Ni_" in filename:
                nmc_type = "Ni_variant"
            elif "NMC111_Mn_" in filename:
                nmc_type = "Mn_variant"
            elif "NMC111_Co_" in filename:
                nmc_type = "Co_variant"
            elif "NMC111_NMC" in filename:
                nmc_type = "High_Ni"
            else:
                nmc_type = "Standard"
            
            if nmc_type not in composition_analysis:
                composition_analysis[nmc_type] = []
            composition_analysis[nmc_type].append({
                'file': cif_file.name,
                'atoms': len(atoms),
                'composition': dict(composition)
            })
            
            # Miller指数の抽出（ファイル名から）
            for miller_str in ["001", "104", "012", "010", "111", "100", "110"]:
                if miller_str in filename:
                    if miller_str not in miller_analysis:
                        miller_analysis[miller_str] = 0
                    miller_analysis[miller_str] += 1
                    break
            
        except Exception as e:
            print(f"ファイル分析エラー: {cif_file.name} - {e}")
    
    # 結果の表示
    print(f"📊 組成別界面構造数:")
    for comp_type, files in composition_analysis.items():
        print(f"  {comp_type}: {len(files)}個")
        for file_info in files[:3]:  # 最初の3個のみ表示
            print(f"    - {file_info['file']} ({file_info['atoms']} atoms)")
    
    print(f"\n📐 Miller指数別界面構造数:")
    for miller, count in miller_analysis.items():
        print(f"  {miller}: {count}個")
    
    return composition_analysis, miller_analysis


def create_batch_visualization_script(interfaces_dir):
    """界面構造の一括可視化スクリプトを生成"""
    script_content = f'''#!/usr/bin/env python3
"""
NMC-Al界面構造一括可視化スクリプト
自動生成日: {time.strftime('%Y-%m-%d %H:%M:%S')}
"""

import matplotlib.pyplot as plt
from ase.io import read
from ase.visualize.plot import plot_atoms
from pathlib import Path
import numpy as np

def visualize_interfaces():
    interfaces_dir = Path(r"{interfaces_dir}")
    cif_files = list(interfaces_dir.glob("*.cif"))
    
    if not cif_files:
        print("CIFファイルが見つかりません")
        return
    
    # 最初の6個の界面構造を可視化
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.flatten()
    
    for i, cif_file in enumerate(cif_files[:6]):
        try:
            atoms = read(str(cif_file))
            
            # 側面図を描画
            plot_atoms(atoms, axes[i], radii=0.5, rotation=('90x,0y,0z'))
            axes[i].set_title(f"{{cif_file.stem[:30]}}")
            axes[i].set_aspect('equal')
            
        except Exception as e:
            print(f"可視化エラー: {{cif_file.name}} - {{e}}")
            axes[i].text(0.5, 0.5, f'エラー\\n{{cif_file.stem}}', 
                        ha='center', va='center', transform=axes[i].transAxes)
    
    plt.tight_layout()
    plt.savefig(interfaces_dir / "interface_overview.png", dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"可視化完了: {{interfaces_dir / 'interface_overview.png'}}")

if __name__ == "__main__":
    visualize_interfaces()
'''
    
    script_path = Path(interfaces_dir) / "visualize_interfaces.py"
    with open(script_path, 'w', encoding='utf-8') as f:
        f.write(script_content)
    
    print(f"📊 可視化スクリプトを生成しました: {script_path}")
    return script_path


# # =================== 使用例とテスト関数 ===================

# def run_quick_test():
#     """システムの動作確認用クイックテスト"""
#     print("🧪 統合システム動作確認テスト開始")
#     print("-" * 50)
    
#     try:
#         # テスト用システム初期化
#         test_system = IntegratedNMCInterfaceSystem()
        
#         # 基本NMC構造の生成テスト
#         print("1. NMC構造生成テスト...")
#         base_nmc = test_system.create_nmc_structure()
#         print(f"   ✅ NMC構造: {len(base_nmc)} atoms")
        
#         # Al構造生成テスト
#         print("2. Al構造生成テスト...")
#         al_slab = test_system.prepare_al_slab(size=(3,3,5))
#         print(f"   ✅ Al構造: {len(al_slab)} atoms")
        
#         # 組成変動の小規模テスト
#         print("3. 組成変動テスト...")
#         test_composition = {"Ni": 0.6, "Mn": 0.2, "Co": 0.2}
#         variant = test_system.create_structure_from_target(
#             base_nmc, test_composition, ["Ni", "Mn", "Co"])
#         if variant:
#             print(f"   ✅ 組成変動: {len(variant)} atoms")
        
#         print("✅ 全テスト通過")
#         return True
        
#     except Exception as e:
#         print(f"❌ テスト失敗: {e}")
#         return False


# if __name__ == "__main__":
#     # 通常実行
#     main()
    
#     # オプション: テストモードでの実行
#     # run_quick_test()

✅ PFP/Matlantis計算器を正常に初期化しました


In [2]:

import glob
import os
import numpy as np
from ase.io import read, write
from ase.build import fcc100
# from ase.calculators.emt import EMT # Matlantisを使用するため不要
# from ase.optimize import BFGS # Matlantisを使用するため不要

# --- Matlantis モジュールのインポート ---
try:
    from matlantis_features.atoms import MatlantisAtoms
    from matlantis_features.features.common.opt import FireLBFGSASEOptFeature
    from matlantis_features.utils.calculators import pfp_estimator_fn
    from pfp_api_client.pfp.estimator import EstimatorCalcMode
    MATLANTIS_AVAILABLE = True
    print("✅ Matlantis (matlantis_features) モジュールを正常にインポートしました。")
    
except ImportError as e:
    print(f"⚠️ Matlantis モジュールのインポートに失敗しました: {e}")
    print("   Matlantisが利用できないため、スクリプトを終了します。")
    print("   PFP/Matlantisがインストールされている環境で実行してください。")
    MATLANTIS_AVAILABLE = False
    # Matlantisが必須のため、ここで終了する
    exit()

# ----------------------------------------

# --- 1. ファイルリストの取得 ---
file_list = glob.glob('Interface_AlF3*.xyz')

# 除外するファイル
skip_file = ""

print(f"ASE スクリプト開始: {len(file_list)} 個のファイルを検出しました。")
print(f"除外対象: {skip_file}\n")

# --- 2. ファイルのループ処理 ---
for filepath in file_list:
    filename = os.path.basename(filepath)

    # 2. (例外処理) 特定のファイル名をスキップ
    if filename == skip_file:
        print(f"--- スキップ (除外リスト): {filename} ---")
        continue

    print(f"--- 処理開始: {filename} ---")

    try:
        # --- 3. NMC構造の抽出 ---
        atoms = read(filepath)
        
        nmc_indices = [a.index for a in atoms if a.symbol not in ('Al', 'F')]
        
        if not nmc_indices:
            print(f"警告: {filename} にAl/F以外の原子が見つかりません。スキップします。")
            continue
            
        nmc_atoms = atoms[nmc_indices]
        print(f"  [3] NMC構造を抽出 (原子数: {len(nmc_atoms)})")

        # --- 4. NMC構造の保存 ---
        if '_on_NMC_' not in filename:
            print(f"警告: {filename} の形式が不正です ('_on_NMC_' が含まれません)。スキップします。")
            continue
            
        nmc_filename_part = filename.split('_on_NMC_')[1]
        
        output_dir_nmc = 'NMC'
        os.makedirs(output_dir_nmc, exist_ok=True)
        
        nmc_save_path = os.path.join(output_dir_nmc, nmc_filename_part)
        write(nmc_save_path, nmc_atoms)
        print(f"  [4] NMC構造を保存: {nmc_save_path}")

        # --- 5. Alスラブの構築 ---# --- 5. Alスラブの構築 ---
        cell_xy = nmc_atoms.get_cell()[:2]
        
        if np.all(cell_xy == 0):
            print(f"警告: {filename} (NMC) にセル情報がありません。スキップします。")
            continue
        
        # 5a. Al(100) の (1,1,1) プリミティブセルのXYサイズを取得
        #    （Nx, Ny を計算するための基準として）
        try:
            prim_cell = fcc100('Al', size=(1, 1, 1), vacuum=0.0).get_cell()
            prim_a_len = np.linalg.norm(prim_cell[0])
            prim_b_len = np.linalg.norm(prim_cell[1])
        except Exception: # Alがfcc100で作れないなどのエラーをキャッチ
            prim_a_len = 2.86 # フォールバック値 (Alの(100)面の格子定数 approx)
            prim_b_len = 2.86
            print("  [5a] 警告: Al(100) プリミティブセルの取得に失敗。デフォルト値 (2.86 Å) を使用します。")
        
        if prim_a_len < 1e-6 or prim_b_len < 1e-6:
            print(f"警告: Alプリミティブスラブのセルサイズが不正です。スキップします。")
            continue
        
        # 5b. ターゲットセル (NMC) のXYサイズを取得
        target_a_len = np.linalg.norm(cell_xy[0])
        target_b_len = np.linalg.norm(cell_xy[1])
        
        # 5c. ターゲットセルをカバーするために必要な最小の繰り返し回数を計算 (切り上げ)
        Nx = int(np.ceil(target_a_len / prim_a_len))
        Ny = int(np.ceil(target_b_len / prim_b_len))
        
        # 0回はありえないので、最低1回にする
        if Nx == 0: Nx = 1
        if Ny == 0: Ny = 1
        
        print(f"  [5c] Alスラブのサイズを ({Nx} x {Ny} x5層) に決定。")
        
        # 5d. 計算した (Nx, Ny, 10) のサイズで Al スラブを生成
        #     これで new_cell の大きさをカバーできる、原子が詰まったスラブが作られる
        al_slab = fcc100('Al', size=(Nx, Ny, 5), vacuum=0.0)
        
        # 5e. 最終的なセルを定義
        #     Z方向の厚さは、生成した (Nx, Ny, 10) スラブの厚さをそのまま使用
        original_al_thickness = al_slab.get_cell()[2, 2] 
        
        new_cell = np.zeros((3, 3))
        new_cell[:2] = cell_xy           # XYはNMCに合わせる
        new_cell[2, 2] = original_al_thickness # ZはAl 10層の厚さ
        
        # 5f. 拡張したスラブ (al_slab) を、NMCのセル (new_cell) に合わせて
        #     わずかに変形(スケール)させる
        #     (Nx, Ny) >= ターゲットサイズ のため、わずかな圧縮/引張になる
        al_slab.set_cell(new_cell, scale_atoms=True)
        al_thickness = al_slab.get_cell()[2, 2]
        
        print(f"  [5f] AlスラブをNMCセルに合わせてスケーリング完了 (原子数: {len(al_slab)})")
        # --- 6. NMC構造の配置 ---
        min_z = nmc_atoms.positions[:, 2].min()
        nmc_atoms.translate([0, 0, -min_z])
        nmc_atoms.translate([0, 0, al_thickness+2.0])
        print(f"  [6] NMCのz座標を調整 (新しい最下部: {nmc_atoms.positions[:, 2].min():.3f} Å)")

        # --- 7. 界面構造の結合 ---
        final_structure = al_slab + nmc_atoms
        print(f"  [7] AlとNMCを結合 (総原子数: {len(final_structure)})")

        # --- 8. 最終構造の設定 ---
        final_structure.set_cell(nmc_atoms.get_cell())
        final_structure.set_pbc([True, True, False])
        final_structure.center(vacuum=10.0, axis=2)
        print(f"  [8] 最終セル設定完了 (PBC: {final_structure.pbc})")

        # --- 9. 構造最適化 (Matlantis) ---
        print(f"  [9] 構造最適化 (Matlantis v7.0.0) を開始...")
        try:
            # MatlantisAtoms に変換
            matlantis_atoms = MatlantisAtoms(final_structure)

            # Estimator関数を定義
            estimator_function = pfp_estimator_fn(
                model_version='v7.0.0',
                calc_mode=EstimatorCalcMode.CRYSTAL_U0_PLUS_D3
            )

            # オプティマイザを初期化
            matlantis_optimizer = FireLBFGSASEOptFeature(
                estimator_fn=estimator_function,
                n_run=3000,
                fmax=0.01,  # ご指定の厳密な条件
                show_progress_bar=True # ターミナルに進捗を表示
            )

            # 最適化を実行
            result = matlantis_optimizer(matlantis_atoms)
            
            # 結果をASE Atomsに戻す
            optimized_atoms = result.atoms.ase_atoms
            final_energy = result.output.energy_log[-1]

            print(f"  [9] ✔️ Matlantis最適化完了。最終エネルギー: {final_energy:.3f} eV")
            
            # 最終構造を最適化されたものに更新
            final_structure = optimized_atoms

        except Exception as e:
            print(f"*** エラー: Matlantisでの最適化中に問題が発生しました: {e} ***")
            print("     最適化されていない構造で保存を試みます。")
            # final_structure は最適化前のものが使われる
        
        # --- 10. 最終ファイルの保存 ---
        output_dir_final = "Al_NMC2"
        
        # ディレクトリが存在しない場合に作成 (存在していてもエラーにならない)
        os.makedirs(output_dir_final, exist_ok=True)
        # final_filename = f"./Al_NMC/Interface_Al_on_NMC_{nmc_filename_part}"
        base_nmc_part = os.path.splitext(nmc_filename_part)[0]
        cif_part_name = f"Interface_Al_on_NMC_{base_nmc_part}.cif"
        final_filename = os.path.join(output_dir_final, cif_part_name)
        write(final_filename, final_structure)
        print(f"  [10] 最終構造を保存: {final_filename}")
        print(f"--- 処理完了: {filename} ---\n")

    except Exception as e:
        print(f"*** エラー発生: {filename} の処理中に問題が発生しました ***")
        print(f"    エラー詳細: {e}")
        print(f"--- スキップ (エラー): {filename} ---\n")

print("すべての処理が完了しました。")


✅ Matlantis (matlantis_features) モジュールを正常にインポートしました。
ASE スクリプト開始: 40 個のファイルを検出しました。
除外対象: 

--- 処理開始: Interface_AlF3_on_NMC_Ni_decrease_010.xyz ---
  [3] NMC構造を抽出 (原子数: 434)
  [4] NMC構造を保存: NMC/Ni_decrease_010.xyz
  [5c] Alスラブのサイズを (5 x 5 x 10層) に決定。
  [5f] AlスラブをNMCセルに合わせてスケーリング完了 (原子数: 125)
  [6] NMCのz座標を調整 (新しい最下部: 10.100 Å)
  [7] AlとNMCを結合 (総原子数: 559)
  [8] 最終セル設定完了 (PBC: [ True  True False])
  [9] 構造最適化 (Matlantis v7.0.0) を開始...


  0%|          | 0/3001 [00:00<?, ?it/s]

  [9] ✔️ Matlantis最適化完了。最終エネルギー: -2486.677 eV
  [10] 最終構造を保存: Al_NMC2/Interface_Al_on_NMC_Ni_decrease_010.cif
--- 処理完了: Interface_AlF3_on_NMC_Ni_decrease_010.xyz ---

--- 処理開始: Interface_AlF3_on_NMC_Mn_decrease_104.xyz ---
  [3] NMC構造を抽出 (原子数: 437)
  [4] NMC構造を保存: NMC/Mn_decrease_104.xyz
  [5c] Alスラブのサイズを (5 x 5 x 10層) に決定。
  [5f] AlスラブをNMCセルに合わせてスケーリング完了 (原子数: 125)
  [6] NMCのz座標を調整 (新しい最下部: 10.100 Å)
  [7] AlとNMCを結合 (総原子数: 562)
  [8] 最終セル設定完了 (PBC: [ True  True False])
  [9] 構造最適化 (Matlantis v7.0.0) を開始...


  0%|          | 0/3001 [00:00<?, ?it/s]

  [9] ✔️ Matlantis最適化完了。最終エネルギー: -2481.243 eV
  [10] 最終構造を保存: Al_NMC2/Interface_Al_on_NMC_Mn_decrease_104.cif
--- 処理完了: Interface_AlF3_on_NMC_Mn_decrease_104.xyz ---

--- 処理開始: Interface_AlF3_on_NMC_NMC811_approx_001.xyz ---
  [3] NMC構造を抽出 (原子数: 460)
  [4] NMC構造を保存: NMC/NMC811_approx_001.xyz
  [5c] Alスラブのサイズを (5 x 5 x 10層) に決定。
  [5f] AlスラブをNMCセルに合わせてスケーリング完了 (原子数: 125)
  [6] NMCのz座標を調整 (新しい最下部: 10.100 Å)
  [7] AlとNMCを結合 (総原子数: 585)
  [8] 最終セル設定完了 (PBC: [ True  True False])
  [9] 構造最適化 (Matlantis v7.0.0) を開始...


...

  [9] ✔️ Matlantis最適化完了。最終エネルギー: -2583.793 eV
  [10] 最終構造を保存: Al_NMC2/Interface_Al_on_NMC_Ni_increase_104.cif
--- 処理完了: Interface_AlF3_on_NMC_Ni_increase_104.xyz ---

--- 処理開始: Interface_AlF3_on_NMC_NMC622_approx_012.xyz ---
  [3] NMC構造を抽出 (原子数: 390)
  [4] NMC構造を保存: NMC/NMC622_approx_012.xyz
  [5c] Alスラブのサイズを (5 x 5 x 10層) に決定。
  [5f] AlスラブをNMCセルに合わせてスケーリング完了 (原子数: 125)
  [6] NMCのz座標を調整 (新しい最下部: 10.100 Å)
  [7] AlとNMCを結合 (総原子数: 515)
  [8] 最終セル設定完了 (PBC: [ True  True False])
  [9] 構造最適化 (Matlantis v7.0.0) を開始...


  0%|          | 0/3001 [00:00<?, ?it/s]

  [9] ✔️ Matlantis最適化完了。最終エネルギー: -2267.560 eV
  [10] 最終構造を保存: Al_NMC2/Interface_Al_on_NMC_NMC622_approx_012.cif
--- 処理完了: Interface_AlF3_on_NMC_NMC622_approx_012.xyz ---

--- 処理開始: Interface_AlF3_on_NMC_Mn_increase_104.xyz ---
  [3] NMC構造を抽出 (原子数: 449)
  [4] NMC構造を保存: NMC/Mn_increase_104.xyz
  [5c] Alスラブのサイズを (5 x 5 x 10層) に決定。
  [5f] AlスラブをNMCセルに合わせてスケーリング完了 (原子数: 125)
  [6] NMCのz座標を調整 (新しい最下部: 10.100 Å)
  [7] AlとNMCを結合 (総原子数: 574)
  [8] 最終セル設定完了 (PBC: [ True  True False])
  [9] 構造最適化 (Matlantis v7.0.0) を開始...


  0%|          | 0/3001 [00:00<?, ?it/s]

  [9] ✔️ Matlantis最適化完了。最終エネルギー: -2558.555 eV
  [10] 最終構造を保存: Al_NMC2/Interface_Al_on_NMC_Mn_increase_104.cif
--- 処理完了: Interface_AlF3_on_NMC_Mn_increase_104.xyz ---

すべての処理が完了しました。


In [9]:
def find_and_categorize_surfaces(self):
        """
        表面ファイルの分類 (修正版)
        - output_dir から最適化済みファイル (*_final.xyz) を検索
        - structure_dir (一段上) から既存の .xyz ファイル (NMC, AlF3, Al2O3...) を検索
        """
        print(f"🔍 表面ファイルを分類中...")
        
        # 検索ディレクトリを定義
        output_dir = self.output_dir
        structure_dir = self.base_dir / "structure"
        print(f"  -> 検索対象 (最適化済): {output_dir}")
        print(f"  -> 検索対象 (既存XYZ): {structure_dir}")

        # --- Al表面 (Alメタル) の検索 ---
        # Alメタルスラブは `output_dir` にある `*_final.xyz` (Al_ で始まる) と仮定
        al_files_from_output = [
            p for p in output_dir.glob("*_final.xyz") 
            if p.stem.startswith("Al_")
        ]
        # Alメタルスラブは `structure_dir` にある `*.xyz` (Al_ で始まる) も検索
        al_files_from_structure = [
            p for p in structure_dir.glob("*.xyz") 
            if p.stem.startswith("Al_") and \
               p.is_file() and p.parent == structure_dir
        ]
        
        al_surfaces = list(set(al_files_from_output + al_files_from_structure))
        print(f"  -> [Alメタル] (Al_...): {len(al_surfaces)}個 発見")


        # --- 正極材表面 (NMC, AlF3, Al2O3など) の検索 ---
        
        # 検索するプレフィックス (Alメタルは除外)
        cathode_prefixes = ["NMC"]

        # 1. `output_dir` から検索 (最適化済みファイル)
        cathode_files_from_output = []
        for prefix in cathode_prefixes:
            cathode_files_from_output.extend(
                output_dir.glob(f"{prefix}*_final.xyz")
            )
        
        print(f"  -> [正極材] (from {output_dir}): {len(cathode_files_from_output)}個 発見")

        # 2. `structure_dir` から検索 (ユーザー指定の既存ファイルパス)
        cathode_files_from_structure = []
        for prefix in cathode_prefixes:
            files = [
                p for p in structure_dir.glob(f"{prefix}*.xyz")
                if p.is_file() and p.parent == structure_dir # サブディレクトリを除外
            ]
            cathode_files_from_structure.extend(files)

        print(f"  -> [正極材] (from {structure_dir}): {len(cathode_files_from_structure)}個 発見")
        
        # 3. リストを結合して重複を削除
        all_cathode_files = list(set(cathode_files_from_output + cathode_files_from_structure))
        cathode_surfaces = all_cathode_files
        
        print(f"  ---")
        print(f"  ✅ [最終] Al表面 (Al_...): {len(al_surfaces)}個")
        print(f"  ✅ [最終] 正極材表面 (NMC/AlF3/Al2O3...): {len(cathode_surfaces)}個")
        
        return al_surfaces, cathode_surfaces

In [7]:
print("--- 1. 統合システムの初期化 ---")
try:
    # クラス定義が読み込まれていると仮定
    system = IntegratedNMCInterfaceSystem()
    print(f"✅ システム初期化完了。")
    print(f"   出力先: {system.output_dir}")
    print(f"   界面保存先: {system.interfaces_dir}")
except NameError:
    print("❌ エラー: IntegratedNMCInterfaceSystem が定義されていません。")
    print("   NMC.ipynb のクラス定義セルを先に実行してください。")
except Exception as e:
    print(f"❌ システム初期化エラー: {e}")

--- 1. 統合システムの初期化 ---
✅ 統合システム初期化完了
  ベースディレクトリ: /home/jovyan/Kaori/MD/LiB_2
  出力ディレクトリ: /home/jovyan/Kaori/MD/LiB_2/structure/output
  界面保存先: /home/jovyan/Kaori/MD/LiB_2/structure/output/final_interfaces2
✅ システム初期化完了。
   出力先: /home/jovyan/Kaori/MD/LiB_2/structure/output
   界面保存先: /home/jovyan/Kaori/MD/LiB_2/structure/output/final_interfaces2


In [10]:
from ase.build import fcc100
from pathlib import Path

print("\n--- 2. Al(100)スラブ (6,6,10) の新規作成 ---")

if 'system' in locals():
    al_slab_name = "Al_100_6x6x10" # ファイル名
    traj_path = system.output_dir / f"{al_slab_name}_optimization.traj"
    # cleanup_trajectory_files が最終的に作成するファイル名
    final_xyz_path = system.output_dir / f"{al_slab_name}_optimization_final.xyz"

    if final_xyz_path.exists():
        print(f"✅ 既存の最適化済みファイルあり、スキップ: {final_xyz_path.name}")
    elif traj_path.exists():
        print(f"✅ 既存のTrajectoryファイルあり、スキップ (セル3で処理): {traj_path.name}")
    else:
        try:
            # 1. Al(100)スラブを作成 (ASEのfcc100を使用)
            # これがご要望の (6, 6, 10) サイズです
            al_slab_atoms = fcc100("Al", size=(6, 6, 10), orthogonal=True, periodic=True, vacuum=5.0)
            print(f"  -> Al(100)スラブ作成: {len(al_slab_atoms)} atoms")
        
            # 2. 構造の最適化 (Matlantis)
            # .traj ファイル (と .xyz ファイル) が保存される
            optimized_al_slab = system.run_matlantis_optimization(
                al_slab_atoms,
                trajectory_path=traj_path,
                fmax=0.05,
                name=al_slab_name
            )
        
            if optimized_al_slab is not None:
                print(f"✅ 最適化完了。Trajectory保存先: {traj_path.name}")
            else:
                print(f"❌ Alスラブの最適化に失敗しました。")

        except Exception as e:
            print(f"❌ Alスラブ作成中にエラー: {e}")
else:
    print("❌ 'system' が初期化されていません。セル 1 を実行してください。")


--- 2. Al(100)スラブ (6,6,10) の新規作成 ---
  -> Al(100)スラブ作成: 360 atoms
  -> Matlantis最適化開始 (Al_100_6x6x10, fmax = 0.05) ...


  0%|          | 0/5001 [00:00<?, ?it/s]

  -> ✔️ 最適化完了！ エネルギー: -1302.199 eV
✅ 最適化完了。Trajectory保存先: Al_100_6x6x10_optimization.traj


In [12]:
def build_interface_with_stack(self, slab_bottom, slab_top, separation=2.5):
        """
        ASEのstack機能を使って2つのスラブから界面を構築する（修正版）
        - Al (slab_bottom) が下 (z=0付近) になるように配置
        - NMC (slab_top) が上に積層される
        - XYセルサイズは大きい方に自動的に合わせられる
        """
        try:
            print(f"    -> ASE.build.stack を使用 (界面距離: {separation} Å)")
            
            # 1. 各スラブのZ座標を調整 (底面がZ=0になるように)
            #    stackはスラブの底面を自動で合わせないので、手動で調整
            pos_bottom = slab_bottom.get_positions()
            pos_bottom[:, 2] -= pos_bottom[:, 2].min()
            slab_bottom.set_positions(pos_bottom)
            
            pos_top = slab_top.get_positions()
            pos_top[:, 2] -= pos_top[:, 2].min()
            slab_top.set_positions(pos_top)

            # 2. 2つのスラブをZ軸(axis=2)に沿ってスタック
            #    distance = 界面間の原子"表面"間の距離
            interface = stack(slab_bottom, slab_top, axis=2, distance=separation)
            
            # 3. セルとPBC（周期境界条件）の設定
            #    stack() はXYのセルを自動で最大値に設定します
            #    Z方向のセルは、(slab1_z_height + slab2_z_height + distance) になります
            #    真空層をさらに追加
            interface.center(vacuum=15.0, axis=2)
            
            # Z方向は真空層があるのでPBCをFalseに
            interface.pbc = (True, True, False)
            
            print(f"    -> スタック完了 (原子数: {len(interface)})")
            cell_dims = interface.get_cell_lengths_and_angles()
            print(f"    -> 新しいセルサイズ (Å): [X: {cell_dims[0]:.2f}, Y: {cell_dims[1]:.2f}, Z: {cell_dims[2]:.2f}]")
            
            return interface

        except Exception as e:
            print(f"    -> ❌ ASE.build.stack でエラーが発生しました: {e}")
            import traceback
            traceback.print_exc()
            return None

In [17]:
def create_trim_and_optimize_interface(self, slab1_path, slab2_path):
        """
        界面の作成、最適化 (スタック機能版に修正)
        - slab1 (Al) と slab2 (NMC) をスタック
        - ハードコードされたトリミングを廃止
        """
        # ファイル名からAlスラブとNMCスラブの名前を特定
        slab1_name_raw = slab1_path.stem.replace('_optimization_final', '')
        slab2_name_raw = slab2_path.stem.replace('_optimization_final', '')
        
        # AlF3_... や Al_100... への対応
        if "Al_" in slab1_name_raw:
            al_slab_name = slab1_name_raw
            cathode_name = slab2_name_raw
            # slab1(Al)が下、slab2(NMC)が上
            slab_bottom_path = slab1_path
            slab_top_path = slab2_path
        else:
            al_slab_name = slab2_name_raw
            cathode_name = slab1_name_raw
            # slab2(Al)が下、slab1(NMC)が上
            slab_bottom_path = slab2_path
            slab_top_path = slab1_path

        interface_name = f"Interface_{cathode_name}_on_{al_slab_name}"
        output_path = self.interfaces_dir / f"{interface_name}.cif"

        if output_path.exists():
            print(f"✅ 既存ファイルあり、スキップ: {output_path.name}")
            return

        print(f"\n--- 処理開始: {cathode_name} (上) + {al_slab_name} (下) ---")
        try:
            # STEP 1: 構築 (新しいスタック関数を使用)
            slab_bottom = read(str(slab_bottom_path))
            slab_top = read(str(slab_top_path))
            
            # slab1(Al) が下、slab2(NMC) が上になるようにスタック
            interface_stacked = self.build_interface_with_stack(
                slab_bottom, 
                slab_top, 
                separation=2.5 # Al-O間の典型的な距離
            )

            if interface_stacked is None:
                print("    -> ❌ スタックに失敗しました。")
                return

            # STEP 2: 最適化 (古いtrim_interfaceはスキップ)
            print(f"    2. スタック後の構造を最適化中...")
            traj_path = self.interfaces_dir / f"{interface_name}.traj"
            optimized_structure = self.run_matlantis_optimization(
                interface_stacked, traj_path, fmax=0.05, name=interface_name)

            # STEP 3: 保存
            if optimized_structure:
                # 念のため最終的なセルサイズとPBCを再設定
                optimized_structure.center(vacuum=15.0, axis=2)
                optimized_structure.pbc = (True, True, False)
                
                print(f"    3. 最終構造をファイルに保存中 -> {output_path.name}")
                write(str(output_path), optimized_structure)
                print(f"✅ 界面構造完成: {interface_name}")
            else:
                print("    -> ❌ 最適化に失敗しました。")

        except Exception as e:
            print(f"    -> ❌ この界面の作成中にエラーが発生: {e}")

In [29]:
# ASE
import ase
from ase import Atoms, units
from ase.io import read, write
from ase.build import surface, molecule, add_adsorbate, fcc100, stack  # <--- 'stack' を追加
from ase.constraints import FixAtoms, FixedPlane, FixBondLength, ExpCellFilter, UnitCellFilter
from ase.optimize import BFGS, LBFGS, FIRE
from ase.visualize import view
def __init__(self, base_dir="/home/jovyan/Kaori/MD/LiB_2"):
        self.base_dir = Path(base_dir)
        self.input_dir = Path("/home/jovyan/Kaori/MD/input")
        self.output_dir = self.base_dir / "structure" / "output"
        self.interfaces_dir = self.output_dir / "final_interfaces2"
        
        # ディレクトリ作成
        for dir_path in [self.input_dir, self.output_dir, self.interfaces_dir]:
            dir_path.mkdir(parents=True, exist_ok=True)
        
        # 計算器設定
        self.calculator = calculator
        
        # Miller指数リスト（重要度順）
        self.miller_indices = [
            (0, 0, 1),  # NMC(001) - 最安定面、プレート形状
            (1, 0, 4),  # NMC(104) - 中活性、多面体形状
            (0, 1, 2),  # NMC(012) - 高活性、切頭八面体形状
            (0, 1, 0),  # NMC(010) - Li拡散活性面
            (1, 1, 1),  # Al(111) - Al最安定面
            (1, 0, 0),  # Al(100), NMC(100) - 基本面
            (1, 1, 0),  # Al(110), NMC(110) - 副次面
        ]
        
        print(f"✅ 統合システム初期化完了")
        print(f"  ベースディレクトリ: {self.base_dir}")
        print(f"  出力ディレクトリ: {self.output_dir}")
        print(f"  界面保存先: {self.interfaces_dir}")
    
def find_and_categorize_surfaces(self):
    """
    表面ファイルの分類 (修正版)
    - output_dir から最適化済みファイル (*_final.xyz) を検索
    - structure_dir (一段上) から既存の .xyz ファイル (NMC, AlF3, Al2O3...) を検索
    """
    print(f"🔍 表面ファイルを分類中...")
    
    # 検索ディレクトリを定義
    output_dir = self.output_dir
    structure_dir = self.base_dir / "structure" # 'structure' ディレクトリ
    print(f"  -> 検索対象 (最適化済): {output_dir}")
    print(f"  -> 検索対象 (既存XYZ): {structure_dir}")

    # --- Al表面 (Alメタル) の検索 ---
    # "Al_" (アンダースコア付き) で始まるファイルのみをAlメタルとして扱う
    al_files_from_output = [
        p for p in output_dir.glob("*_final.xyz") 
        if p.stem.startswith("Al_")
    ]
    al_files_from_structure = [
        p for p in structure_dir.glob("*.xyz") 
        if p.stem.startswith("Al_") and \
           p.is_file() and p.parent == structure_dir
    ]
    al_surfaces = list(set(al_files_from_output + al_files_from_structure))
    print(f"  -> [Alメタル] (Al_...): {len(al_surfaces)}個 発見")

    # --- 正極材表面 (NMC, AlF3, Al2O3など) の検索 ---
    cathode_prefixes = ["NMC_" ]

    # 1. `output_dir` から検索 (最適化済みファイル)
    cathode_files_from_output = []
    directory=self.base_dir / "structure"
    for prefix in cathode_prefixes:
        cathode_files_from_output.extend(
            directory.glob(f"{prefix}*.xyz")
        )
    print(f"  -> [正極材] (from {output_dir}): {len(cathode_files_from_output)}個 発見")

    # 2. `structure_dir` から検索 (ユーザー指定の既存ファイルパス)
    cathode_files_from_structure = []
    for prefix in cathode_prefixes:
        files = [
            p for p in structure_dir.glob(f"{prefix}*.xyz")
            if p.is_file() and p.parent == structure_dir # サブディレクトリを除外
        ]
        cathode_files_from_structure.extend(files)
    print(f"  -> [正極材] (from {structure_dir}): {len(cathode_files_from_structure)}個 発見")
    
    # 3. リストを結合して重複を削除
    cathode_surfaces = list(set(cathode_files_from_output + cathode_files_from_structure))
    
    print(f"  ---")
    print(f"  ✅ [最終] Al表面 (Al_...): {len(al_surfaces)}個")
    print(f"  ✅ [最終] 正極材表面 (NMC/AlF3/Al2O3...): {len(cathode_surfaces)}個")
    
    return al_surfaces, cathode_surfaces
def build_interface_with_stack(self, slab_bottom, slab_top, separation=2.5):
        """
        ASEのstack機能を使って2つのスラブから界面を構築する（修正版）
        - Al (slab_bottom) が下 (z=0付近) になるように配置
        - NMC (slab_top) が上に積層される
        - XYセルサイズは大きい方に自動的に合わせられる
        """
        try:
            print(f"    -> ASE.build.stack を使用 (界面距離: {separation} Å)")
            
            # 1. 各スラブのZ座標を調整 (底面がZ=0になるように)
            pos_bottom = slab_bottom.get_positions()
            pos_bottom[:, 2] -= pos_bottom[:, 2].min()
            slab_bottom.set_positions(pos_bottom)
            
            pos_top = slab_top.get_positions()
            pos_top[:, 2] -= pos_top[:, 2].min()
            slab_top.set_positions(pos_top)

            # 2. 2つのスラブをZ軸(axis=2)に沿ってスタック
            interface = stack(slab_bottom, slab_top, axis=2, distance=separation)
            
            # 3. セルとPBC（周期境界条件）の設定
            interface.center(vacuum=15.0, axis=2)
            interface.pbc = (True, True, False)
            
            print(f"    -> スタック完了 (原子数: {len(interface)})")
            cell_dims = interface.get_cell_lengths_and_angles()
            print(f"    -> 新しいセルサイズ (Å): [X: {cell_dims[0]:.2f}, Y: {cell_dims[1]:.2f}, Z: {cell_dims[2]:.2f}]")
            
            return interface

        except Exception as e:
            print(f"    -> ❌ ASE.build.stack でエラーが発生しました: {e}")
            import traceback
            traceback.print_exc()
            return None
def create_trim_and_optimize_interface(self, slab1_path, slab2_path):
        """
        界面の作成、最適化 (スタック機能版・簡略化)
        - slab1 (Al) と slab2 (NMC/Cathode) をスタック
        - (process_all_interfacesから渡されるパスを信頼する)
        """
        
        # slab1_path = al_path (Alメタル)
        # slab2_path = cathode_path (NMC, AlF3など)
        slab_bottom_path = slab1_path
        slab_top_path = slab2_path
        
        # ファイル名から名前をクリーンアップ
        al_slab_name = slab_bottom_path.stem.replace('_optimization_final', '').replace('_final', '')
        cathode_name = slab_top_path.stem.replace('_optimization_final', '').replace('_final', '')

        interface_name = f"Interface_{cathode_name}_on_{al_slab_name}"
        output_path = self.interfaces_dir / f"{interface_name}.cif"

        if output_path.exists():
            print(f"✅ 既存ファイルあり、スキップ: {output_path.name}")
            return

        print(f"\n--- 処理開始: {cathode_name} (上) + {al_slab_name} (下) ---")
        try:
            # STEP 1: 構築 (新しいスタック関数を使用)
            slab_bottom = read(str(slab_bottom_path))
            slab_top = read(str(slab_top_path))
            
            # slab_bottom(Al) が下、slab_top(NMC) が上になるようにスタック
            interface_stacked = self.build_interface_with_stack(
                slab_bottom, 
                slab_top, 
                separation=2.5 # Al-O/F間の典型的な距離
            )

            if interface_stacked is None:
                print("    -> ❌ スタックに失敗しました。")
                return

            # STEP 2: 最適化
            print(f"    2. スタック後の構造を最適化中...")
            traj_path = self.interfaces_dir / f"{interface_name}.traj"
            optimized_structure = self.run_matlantis_optimization(
                interface_stacked, traj_path, fmax=0.05, name=interface_name)

            # STEP 3: 保存
            if optimized_structure:
                # 念のため最終的なセルサイズとPBCを再設定
                optimized_structure.center(vacuum=15.0, axis=2)
                optimized_structure.pbc = (True, True, False)
                
                print(f"    3. 最終構造をファイルに保存中 -> {output_path.name}")
                write(str(output_path), optimized_structure)
                print(f"✅ 界面構造完成: {interface_name}")
            else:
                print("    -> ❌ 最適化に失敗しました。")

        except Exception as e:
            print(f"    -> ❌ この界面の作成中にエラーが発生: {e}")

def create_trim_and_optimize_interface(self, slab1_path, slab2_path):
    """
    界面の作成、最適化 (スタック機能版・簡略化)
    - slab1 (Al) と slab2 (NMC/Cathode) をスタック
    - (process_all_interfacesから渡されるパスを信頼する)
    """
    
    # slab1_path = al_path (Alメタル)
    # slab2_path = cathode_path (NMC, AlF3など)
    slab_bottom_path = slab1_path
    slab_top_path = slab2_path
    
    # ファイル名から名前をクリーンアップ
    al_slab_name = slab_bottom_path.stem.replace('_optimization_final', '').replace('_final', '')
    cathode_name = slab_top_path.stem.replace('_optimization_final', '').replace('_final', '')

    interface_name = f"Interface_{cathode_name}_on_{al_slab_name}"
    output_path = self.interfaces_dir / f"{interface_name}.cif"

    if output_path.exists():
        print(f"✅ 既存ファイルあり、スキップ: {output_path.name}")
        return

    print(f"\n--- 処理開始: {cathode_name} (上) + {al_slab_name} (下) ---")
    try:
        # STEP 1: 構築 (新しいスタック関数を使用)
        slab_bottom = read(str(slab_bottom_path))
        slab_top = read(str(slab_top_path))
        
        # slab_bottom(Al) が下、slab_top(NMC) が上になるようにスタック
        interface_stacked = self.build_interface_with_stack(
            slab_bottom, 
            slab_top, 
            separation=2.5 # Al-O/F間の典型的な距離
        )

        if interface_stacked is None:
            print("    -> ❌ スタックに失敗しました。")
            return

        # STEP 2: 最適化
        print(f"    2. スタック後の構造を最適化中...")
        traj_path = self.interfaces_dir / f"{interface_name}.traj"
        optimized_structure = self.run_matlantis_optimization(
            interface_stacked, traj_path, fmax=0.05, name=interface_name)

        # STEP 3: 保存
        if optimized_structure:
            # 念のため最終的なセルサイズとPBCを再設定
            optimized_structure.center(vacuum=15.0, axis=2)
            optimized_structure.pbc = (True, True, False)
            
            print(f"    3. 最終構造をファイルに保存中 -> {output_path.name}")
            write(str(output_path), optimized_structure)
            print(f"✅ 界面構造完成: {interface_name}")
        else:
            print("    -> ❌ 最適化に失敗しました。")

    except Exception as e:
        print(f"    -> ❌ この界面の作成中にエラーが発生: {e}")
def find_and_categorize_surfaces(self):
        """
        表面ファイルの分類 (ユーザー指示反映版 v2)
        - Alメタル ("Al_") は output_dir と structure_dir から検索
        - NMC ("NMC") は structure_dir の *.xyz のみ検索
        """
        print(f"🔍 表面ファイルを分類中... (ユーザー指示反映版 v2)")
        
        output_dir = self.output_dir
        structure_dir = self.base_dir / "structure"
        print(f"  -> 検索対象 (Alメタル最適化済): {output_dir}")
        print(f"  -> 検索対象 (Alメタル/NMC既存): {structure_dir}")

        # --- 1. Al表面 (Alメタル) の検索 ---
        # "Al_" (アンダースコア付き) で始まるファイルのみ
        al_files_from_output = list(output_dir.glob("Al_*_final.xyz"))
        
        al_files_from_structure = [
            p for p in structure_dir.glob("Al_*.xyz") 
            if p.is_file() and p.parent == structure_dir
        ]
        al_surfaces = list(set(al_files_from_output + al_files_from_structure))
        print(f"  ✅ [Alメタル] (Al_...): {len(al_surfaces)}個 発見")

        # --- 2. 正極材表面 (NMC) の検索 ---
        # ユーザー指示: NMCのみ、_final.xyzではない (structure_dirにある)
        cathode_files_from_structure = [
            p for p in structure_dir.glob("NMC*.xyz") # "NMC"で始まる
            if p.is_file() and p.parent == structure_dir # サブディレクトリを除外
        ]
        
        cathode_surfaces = list(set(cathode_files_from_structure))
        
        print(f"  ✅ [正極材表面] (NMC... from {structure_dir}): {len(cathode_surfaces)}個 発見")
        
        # デバッグ用: もしNMCが0個だったら、structure_dirの中身を表示
        if len(cathode_surfaces) == 0 and len(al_surfaces) > 0:
            print(f"  -> [デバッグ] {structure_dir} 内の 'NMC*.xyz' ファイルが見つかりません。")
            try:
                # AlF3やAl2O3など、NMC以外も含むリスト
                all_xyz = [
                    p for p in structure_dir.glob("*.xyz") 
                    if p.is_file() and p.parent == structure_dir
                ]
                print(f"  -> [デバッグ] {structure_dir} 内の全.xyzファイル ({len(all_xyz)}個):")
                for i, f in enumerate(all_xyz[:15]): # 最初の15件
                    print(f"     {i+1}. {f.name}")
            except Exception as e:
                print(f"  -> [デバッグ] {structure_dir} のスキャンエラー: {e}")
        
        return al_surfaces, cathode_surfaces

In [32]:
# --- 1. 統合システムの初期化 ---
print("--- 1. 統合システムの初期化 ---")
try:
    system = IntegratedNMCInterfaceSystem()
    print(f"✅ システム初期化完了。")
    print(f"   出力先: {system.output_dir}")
    print(f"   界面保存先: {system.interfaces_dir}")
except Exception as e:
    print(f"❌ システム初期化エラー: {e}")

--- 1. 統合システムの初期化 ---
✅ 統合システム初期化完了
  ベースディレクトリ: /home/jovyan/Kaori/MD/LiB_2
  出力ディレクトリ: /home/jovyan/Kaori/MD/LiB_2/structure/output
  界面保存先: /home/jovyan/Kaori/MD/LiB_2/structure/output/final_interfaces2
✅ システム初期化完了。
   出力先: /home/jovyan/Kaori/MD/LiB_2/structure/output
   界面保存先: /home/jovyan/Kaori/MD/LiB_2/structure/output/final_interfaces2


In [33]:
# --- 4. 界面構築・最適化 (PHASE 3) ---
print("\n--- 4. 界面構築・最適化 (PHASE 3) ---")

if 'system' in locals():
    # 修正された find_and_categorize_surfaces がここで呼ばれます
    # (Al_100_6x6x10_optimization_final.xyz と 既存のNMC*.xyz が読み込まれるはずです)
    system.process_all_interfaces()
else:
    print("❌ 'system' が初期化されていません。セル 1 を実行してください。")


--- 4. 界面構築・最適化 (PHASE 3) ---

--- STEP 3: 界面構築・トリミング・最適化プロセス開始 ---
🔍 表面ファイルを分類中...
  - Al表面: 20個
  - 正極材表面: 0個
 -> 表面ファイルが不足しているため、スキップします。


In [34]:
import glob
import os
import numpy as np
from ase.io import read, write
from ase.build import fcc100
from ase.calculators.emt import EMT
from ase.optimize import BFGS

# --- 1. ファイルリストの取得 ---
file_list = glob.glob('Interface_AlF3*.xyz')

# 除外するファイル
skip_file = 'Interface_AlF3_on_NMC_pristine_012.xyz'

print(f"ASE スクリプト開始: {len(file_list)} 個のファイルを検出しました。")
print(f"除外対象: {skip_file}\n")

# --- 2. ファイルのループ処理 ---
for filepath in file_list:
    filename = os.path.basename(filepath)

    # 2. (例外処理) 特定のファイル名をスキップ
    if filename == skip_file:
        print(f"--- スキップ (除外リスト): {filename} ---")
        continue

    print(f"--- 処理開始: {filename} ---")

    try:
        # --- 3. NMC構造の抽出 ---
        atoms = read(filepath)
        
        # Al と F 以外の原子インデックスを取得
        nmc_indices = [a.index for a in atoms if a.symbol not in ('Al', 'F')]
        
        if not nmc_indices:
            print(f"警告: {filename} にAl/F以外の原子が見つかりません。スキップします。")
            continue
            
        nmc_atoms = atoms[nmc_indices]
        print(f"  [3] NMC構造を抽出 (原子数: {len(nmc_atoms)})")

        # --- 4. NMC構造の保存 ---
        
        # ファイル名をパースしてNMC部分を取得
        if '_on_NMC_' not in filename:
            print(f"警告: {filename} の形式が不正です ('_on_NMC_' が含まれません)。スキップします。")
            continue
            
        nmc_filename_part = filename.split('_on_NMC_')[1]
        
        # 'NMC' ディレクトリの作成
        output_dir_nmc = 'NMC'
        os.makedirs(output_dir_nmc, exist_ok=True)
        
        nmc_save_path = os.path.join(output_dir_nmc, nmc_filename_part)
        write(nmc_save_path, nmc_atoms)
        print(f"  [4] NMC構造を保存: {nmc_save_path}")

        # --- 5. Alスラブの構築 ---
        
        # NMCのセル情報を取得
        cell_xy = nmc_atoms.get_cell()[:2]
        
        if np.all(cell_xy == 0):
             print(f"警告: {filename} (NMC) にセル情報がありません。スキップします。")
             continue

        # Al(100) 10層スラブを生成
        al_slab = fcc100('Al', size=(1, 1, 10), vacuum=0.0)
        
        # Alスラブの元のz方向の厚さを取得
        original_al_thickness = al_slab.get_cell()[2, 2]
        
        # AlスラブのセルをNMCのXYに合わせ、Zは10層分の厚さを維持
        new_cell = np.zeros((3, 3))
        new_cell[:2] = cell_xy
        new_cell[2, 2] = original_al_thickness
        
        al_slab.set_cell(new_cell, scale_atoms=True)
        
        # スケーリング後のAlスラブの実際の厚さを取得
        al_thickness = al_slab.get_cell()[2, 2]
        print(f"  [5] Al 10層スラブを構築 (厚さ: {al_thickness:.3f} Å)")

        # --- 6. NMC構造の配置 ---
        
        # NMCのz座標の最小値を取得
        min_z = nmc_atoms.positions[:, 2].min()
        
        # NMCの最下部が z=0 になるように移動
        nmc_atoms.translate([0, 0, -min_z])
        
        # Alスラブの厚さ分、NMCをz方向に移動
        nmc_atoms.translate([0, 0, al_thickness])
        print(f"  [6] NMCのz座標を調整 (新しい最下部: {nmc_atoms.positions[:, 2].min():.3f} Å)")

        # --- 7. 界面構造の結合 ---
        final_structure = al_slab + nmc_atoms
        print(f"  [7] AlとNMCを結合 (総原子数: {len(final_structure)})")

        # --- 8. 最終構造の設定 ---
        
        # セルをNMCのXYに合わせる (ZはNMCのものを一旦使う)
        final_structure.set_cell(nmc_atoms.get_cell())
        
        # Z方向を非周期境界に (X, Y は周期境界)
        final_structure.set_pbc([True, True, False])
        
        # Z軸方向に 10Å の真空層を追加してセンタリング
        final_structure.center(vacuum=10.0, axis=2)
        print(f"  [8] 最終セル設定完了 (PBC: {final_structure.pbc})")

        # --- 9. 構造最適化 ---
        print(f"  [9] 構造最適化 (EMT) を開始...")
        final_structure.set_calculator(EMT())
        opt = BFGS(final_structure)
        opt.run(fmax=0.05)
        print(f"  [9] 構造最適化 完了。")

        # --- 10. 最終ファイルの保存 ---
        final_filename = f"Interface_Al_on_NMC_{nmc_filename_part}"
        write(final_filename, final_structure)
        print(f"  [10] 最終構造を保存: {final_filename}")
        print(f"--- 処理完了: {filename} ---\n")

    except Exception as e:
        print(f"*** エラー発生: {filename} の処理中に問題が発生しました ***")
        print(f"    エラー詳細: {e}")
        print(f"--- スキップ (エラー): {filename} ---\n")

print("すべての処理が完了しました。")

ASE スクリプト開始: 40 個のファイルを検出しました。
除外対象: Interface_AlF3_on_NMC_pristine_012.xyz

--- 処理開始: Interface_AlF3_on_NMC_Ni_decrease_010.xyz ---
  [3] NMC構造を抽出 (原子数: 434)
  [4] NMC構造を保存: NMC/Ni_decrease_010.xyz
  [5] Al 10層スラブを構築 (厚さ: 18.225 Å)
  [6] NMCのz座標を調整 (新しい最下部: 18.225 Å)
  [7] AlとNMCを結合 (総原子数: 444)
  [8] 最終セル設定完了 (PBC: [ True  True False])
  [9] 構造最適化 (EMT) を開始...
*** エラー発生: Interface_AlF3_on_NMC_Ni_decrease_010.xyz の処理中に問題が発生しました ***
    エラー詳細: No EMT-potential for Li
--- スキップ (エラー): Interface_AlF3_on_NMC_Ni_decrease_010.xyz ---

--- 処理開始: Interface_AlF3_on_NMC_Mn_decrease_104.xyz ---
  [3] NMC構造を抽出 (原子数: 437)
  [4] NMC構造を保存: NMC/Mn_decrease_104.xyz
  [5] Al 10層スラブを構築 (厚さ: 18.225 Å)
  [6] NMCのz座標を調整 (新しい最下部: 18.225 Å)
  [7] AlとNMCを結合 (総原子数: 447)
  [8] 最終セル設定完了 (PBC: [ True  True False])
  [9] 構造最適化 (EMT) を開始...
*** エラー発生: Interface_AlF3_on_NMC_Mn_decrease_104.xyz の処理中に問題が発生しました ***
    エラー詳細: No EMT-potential for Li
--- スキップ (エラー): Interface_AlF3_on_NMC_Mn_decrease_104.xyz ---

--- 処理開始:

/tmp/ipykernel_2964/2544267953.py:116: FutureWarning: Please use atoms.calc = calc
  final_structure.set_calculator(EMT())


*** エラー発生: Interface_AlF3_on_NMC_NMC523_approx_001.xyz の処理中に問題が発生しました ***
    エラー詳細: No EMT-potential for Li
--- スキップ (エラー): Interface_AlF3_on_NMC_NMC523_approx_001.xyz ---

--- 処理開始: Interface_AlF3_on_NMC_Mn_decrease_012.xyz ---
  [3] NMC構造を抽出 (原子数: 388)
  [4] NMC構造を保存: NMC/Mn_decrease_012.xyz
  [5] Al 10層スラブを構築 (厚さ: 18.225 Å)
  [6] NMCのz座標を調整 (新しい最下部: 18.225 Å)
  [7] AlとNMCを結合 (総原子数: 398)
  [8] 最終セル設定完了 (PBC: [ True  True False])
  [9] 構造最適化 (EMT) を開始...
*** エラー発生: Interface_AlF3_on_NMC_Mn_decrease_012.xyz の処理中に問題が発生しました ***
    エラー詳細: No EMT-potential for Li
--- スキップ (エラー): Interface_AlF3_on_NMC_Mn_decrease_012.xyz ---

--- 処理開始: Interface_AlF3_on_NMC_Co_increase_104.xyz ---
  [3] NMC構造を抽出 (原子数: 433)
  [4] NMC構造を保存: NMC/Co_increase_104.xyz
  [5] Al 10層スラブを構築 (厚さ: 18.225 Å)
  [6] NMCのz座標を調整 (新しい最下部: 18.225 Å)
  [7] AlとNMCを結合 (総原子数: 443)
  [8] 最終セル設定完了 (PBC: [ True  True False])
  [9] 構造最適化 (EMT) を開始...
*** エラー発生: Interface_AlF3_on_NMC_Co_increase_104.xyz の処理中に問題が発生しました ***
    エラー詳細:

In [18]:
print("\n--- 4. 界面構築・最適化 (PHASE 3) ---")

if 'system' in locals():
    # この関数は output_dir の _final.xyz ファイルを自動で検出し
    # 界面構築 -> トリミング -> 最適化 を行います。
    
    # process_all_interfaces -> create_trim_and_optimize_interface -> build_interface
    # の流れで、al_path (slab1) が下、cathode_path (slab2) が上になるように
    # 自動的に処理されます。
    system.process_all_interfaces()
else:
    print("❌ 'system' が初期化されていません。セル 1 を実行してください。")


--- 4. 界面構築・最適化 (PHASE 3) ---

--- STEP 3: 界面構築・トリミング・最適化プロセス開始 ---
🔍 表面ファイルを分類中...
  - Al表面: 20個
  - 正極材表面: 0個
 -> 表面ファイルが不足しているため、スキップします。
